In [1]:
import gymnasium as gym
from squid_GYM_env import trainSquid




In [2]:
# from stable_baselines3.common.env_checker import check_env
# env = trainSquid(level=8)
# print(isinstance(env, gym.Env))
# check_env(env, warn=True)

In [3]:
import numpy as np
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import PPO

def make_env(level, rank):
    def _init():
        env = trainSquid(level=level)
        env.seed(rank)
        return env
    return _init




In [4]:
num_envs = 10  # 根據你的機器性能調整
env = SubprocVecEnv([make_env(8, i) for i in range(num_envs)])

In [8]:
import torch.nn as nn
import torch
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.policies import ActorCriticPolicy

class CustomActorCriticPolicy(ActorCriticPolicy):
    def _build_mlp_extractor(self) -> None:
        # This method replaces the original MLP extractor with your custom version
        self.mlp_extractor = nn.ModuleDict({
            'policy_net': CustomMLPExtractor(self.features_dim, net_arch=[64, 64, 128], activation_fn=nn.ReLU),
            'value_net': CustomMLPExtractor(self.features_dim, net_arch=[64, 64, 128], activation_fn=nn.ReLU)
        })

        self.action_net = nn.Linear(self.mlp_extractor['policy_net'].latent_dim_pi, self.action_space.n)

        # 价值网络 (vf)：输出状态值
        self.value_net = nn.Linear(self.mlp_extractor.latent_dim_vf, 1)

class CustomMLPExtractor(BaseFeaturesExtractor):
    def __init__(self, feature_dim, net_arch, activation_fn):
        super(CustomMLPExtractor, self).__init__(feature_dim, feature_dim)
        self.latent_dim_pi = net_arch[-1]
        self.latent_dim_vf = net_arch[-1]
        modules = []
        last_dim = feature_dim
        for dim in net_arch:
            modules.append(nn.Linear(last_dim, dim))
            modules.append(activation_fn())
            last_dim = dim
        self.net = nn.Sequential(*modules)

    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        return self.net(observations)

In [9]:
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import ProgressBarCallback

env = make_vec_env(trainSquid, n_envs=num_envs, env_kwargs={"level": 8}, wrapper_class=Monitor)
model = PPO(CustomActorCriticPolicy, env, verbose=1, n_steps=4096, n_epochs=20, batch_size=128)
progress_callback = ProgressBarCallback()

print(model.policy)

Using cuda device

AttributeError: 'ModuleDict' object has no attribute 'latent_dim_pi'

In [7]:
model.learn(total_timesteps=5000000, callback=progress_callback)



c:\Users\weiso131\anaconda3\envs\AI\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

ValueError: too many values to unpack (expected 2)

In [ ]:
import torch

checkpoint = model.policy.state_dict()
torch.save(checkpoint, "model07.pth")

In [ ]:
import time


env = trainSquid(level=8, oppoRandom=0.7)
obs, info  = env.reset()
done = False
fight = 500

while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)

    if (terminated or truncated):
        break

    env.render()
    time.sleep(0.01)  # 可調整以控制遊戲速度
env.close()

In [ ]:
print(model.policy)

